# Deep Deterministic Policy Gradient (DDPG)

This implement the pseudo code in [CONTINUOUS CONTROL WITH DEEP REINFORCEMENT
LEARNING](https://arxiv.org/pdf/1509.02971.pdf) from Deepmind in pytorch.
<img src="ddpg_pseudocode.png" width="500"/>

One change in the code here $\mathcal{N}$ is just a uniform random number of -0.5 to 0.5 that decay over number of iteration, $\mathcal{N} = (rand() - 0.5) / (1 + 1e^{-3} * iteration)$. In the paper they used Ornstein-Uhlenbeck process for $\mathcal{N}$, but for simple task like pendulum the uniform random noise seems to work just fine.

## Implementation Detail

The way how DDPG update its Q value function and policy is a quite interesting. The pseudocode is actually pretty detail, though a simple diagram might help visualize the deligate part.

In DDPG it start out with policy(P) and policy_prime(P') with same weight $\theta^\mu$. Same thing for Q, Q(Q) and Q_prime(Q') with $\theta^Q$. Then it sample the environment using policy(P). After it collected enough transitions from the environment, it sample a mini batch from replay memory at each timestep to perform an update. Inside the update process, it calculate the Q loss and policy loss as follow:

<img src="Q_loss.png" width="500"/>  
It use policy_prime and Q_prime to caculate the predicted next-state-action value and use it compute the Q target
$Q\_target=reward + \gamma * Q'(S\_next,P'(S\_next))$  
$Q\_loss = MSE(Q\_target - Q(S,A))$ where $Q\_target$ is using Q' and P' while $Q(S,A)$ just use Q.   
Then do a backprop on Q with $Q\_loss$

For the policy update:  
<img src="ddpg_policy_grad.png" width="350"/>  
Actuall it just mean the gradient of $\theta^\mu$ with respect to the mean of $Q(s,P(s))$. Here in pytorch it performs gradient descent, so the policy loss is $-\overline{Q(s,P(s))}$. It use the policy loss to update P.

After both P and Q are update, do a soft copy of the parameters in the neural network from P to P'. e.g. parameters of P' \* 0.9 + parameters of P \* 0.1. So P will take a full step on the gradient while P' will trail behind it.

In [13]:
%matplotlib inline
from IPython import display
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Normal, Categorical

import numpy as np
import random
import os
import gym

In [14]:
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor

In [15]:
def save_torch_model(model, filename):
  if not os.path.exists(os.path.dirname(filename)):
    os.makedirs(os.path.dirname(filename))
  torch.save(model.state_dict(), filename)

def load_torch_model(model, filename):
  model.load_state_dict(torch.load(filename))

In [16]:
def copy_grad(source, target):
  grads = []
  for param in source.parameters():
    grads.append(param.grad.clone())
  grads.reverse()
  for param in target.parameters():
    param.grad = grads.pop()

def zero_grad(model):
  for param in model.parameters():
    if type(param.grad) != type(None):
      param.grad.data.zero_()
      
def update_target(target_net, eval_net, tau):
  fast = eval_net.state_dict()
  slow = target_net.state_dict()
  for t in slow:
    slow[t] = slow[t] * (1. - tau) + fast[t] * tau

  target_net.load_state_dict(slow)

In [17]:
class ReplayMemory():
  def __init__(self, memory_size = 100000):
    self.transitions = []
    self.memory_size = memory_size
    self.loc_pointer = 0
  
  def clear(self):
    self.transitions = []
    self.loc_pointer = 0
  
  def add(self, step_tuple):
    # expect a tuple of transition contain:
    # state, action, reward, next_state, ended
    if len(self.transitions) <= self.loc_pointer:
      self.transitions.append(None)
    self.transitions[self.loc_pointer] = step_tuple
    self.loc_pointer += 1
    if self.loc_pointer >= self.memory_size:
      self.loc_pointer %= self.memory_size
  
  def get_sample(self, batch_size):
    return random.sample(self.transitions, batch_size)
  
  def usage(self):
    return len(self.transitions) / self.memory_size
  

In [18]:
class PolicyNet_continuous(nn.Module):
  def __init__(self, input_size, output_size):
    super(PolicyNet_continuous,self).__init__()
    self.l1_linear = nn.Linear(input_size, 128)
    self.l2_linear = nn.Linear(128, 64)
    self.l3_linear = nn.Linear(64, output_size)
    nn.init.kaiming_normal_(self.l1_linear.weight)
    nn.init.kaiming_normal_(self.l2_linear.weight)
    self.l3_linear.weight.data.zero_()
    
  def forward(self,x):
    out = F.relu(self.l1_linear(x))
    out = F.relu(self.l2_linear(out))
    out = F.tanh(self.l3_linear(out))
    return out

In [19]:
class QNet(nn.Module):
  def __init__(self, input_size):
    super(QNet,self).__init__()
    self.l1_linear = nn.Linear(input_size, 128)
    self.l2_linear = nn.Linear(128, 64)
    self.l3_linear = nn.Linear(64, 1)
    nn.init.kaiming_normal_(self.l1_linear.weight)
    nn.init.kaiming_normal_(self.l2_linear.weight)
    self.l3_linear.weight.data.zero_()

  def forward(self, state, action):
    out = F.relu(self.l1_linear(torch.cat([state,action],dim=1)))
    out = F.relu(self.l2_linear(out))
    out = self.l3_linear(out)
    return out    

In [90]:
class DDPG():
  def __init__(self, env, steps_in_state = 1):
    self.is_training = True
    self.state_value_range = [{'max':None, 'min':None}] * env.observation_space.shape[0]
    self.replay_memory = ReplayMemory(memory_size=100000)
    self.steps_in_state = steps_in_state
    self.actor = PolicyNet_continuous(env.observation_space.shape[0] * steps_in_state, env.action_space.shape[0])
    self.actor_prime = PolicyNet_continuous(env.observation_space.shape[0] * steps_in_state, env.action_space.shape[0])
    self.critic = QNet(env.observation_space.shape[0] * steps_in_state + env.action_space.shape[0])
    self.critic_prime = QNet(env.observation_space.shape[0] * steps_in_state + env.action_space.shape[0])
    if use_cuda:
      self.actor.cuda()
      self.actor_prime.cuda()
      self.critic.cuda()
      self.critic_prime.cuda()
    # copy the weights from target to eval net
    self.actor_prime.load_state_dict(self.actor.state_dict())
    self.critic_prime.load_state_dict(self.critic.state_dict())
    self.env = env
    self.range_scale = (env.action_space.high[0] - env.action_space.low[0]) / 2.0
    self._gamma = 0.96

  def pick_action(self, state):
    action = self.actor(state)
    action = action.item()
    # add noise
    if self.is_training:
      action += (np.random.rand() - 0.5) / (1 + 1e-3 * self.iteration)
    return np.clip(action, -1.0, 1.0)
  
  def update_ddpg(self, batch):
    (states, actions, rewards, next_states, ended) = zip(*batch)
    states_tensor = torch.stack(states)
    actions_tensor = FloatTensor(actions).view(-1,1)
    rewards_tensor = FloatTensor(rewards).view(-1,1)
    next_states_tensor = torch.stack(next_states)
    ended_tensor = FloatTensor(ended).view(-1,1)

    critic_target = rewards_tensor + self._gamma * (1 - ended_tensor) * \
      self.critic_prime(next_states_tensor, self.actor_prime(next_states_tensor))
    critic_loss = F.mse_loss(self.critic(states_tensor, actions_tensor), critic_target)

    self.critic_coptimizer.zero_grad()
    critic_loss.backward()
    self.critic_coptimizer.step()
    
    actor_loss = -self.critic(states_tensor, self.actor(states_tensor))
    actor_loss = actor_loss.mean()
    
    self.actor_optimizer.zero_grad()
    actor_loss.backward()
    self.actor_optimizer.step()
    
    update_target(self.critic_prime, self.critic, 0.1)
    update_target(self.actor_prime, self.actor, 0.1)
    
  def train(self, env, update_limit=1000, batch_size=200, lr=1e-3, lr_actor=None, lr_critic=None, checkpoint=100):
    lr_actor = lr if lr_actor == None else lr_actor
    lr_critic = lr if lr_critic == None else lr_critic
    
    self.actor_optimizer = torch.optim.SGD(self.actor.parameters(), lr=lr_actor, weight_decay=1e-3)
    self.critic_coptimizer = torch.optim.SGD(self.critic.parameters(), lr=lr_critic, weight_decay=1e-3)
    
    best_score = -99999
    running_score = None
    update_count = 0
    self.iteration = 0
    while update_count < update_limit:
      s0 = env.reset()
      seq = [s0] * self.steps_in_state
      state = FloatTensor(seq).view(-1)
      episode_ended = False
      score = 0
      while not episode_ended:
        action =  self.pick_action(state)
        (s1, reward, episode_ended, info) = env.step([action * self.range_scale])
        seq = seq[1:]
        seq.append(s1)
        next_state = FloatTensor(seq).view(-1)
        if episode_ended:
          ended = 1
        else:
          ended = 0
        self.replay_memory.add((state, action, reward, next_state, ended))
        if self.replay_memory.usage() > 0.8:
          if (update_count + 1) % checkpoint == 0:
            save_torch_model(self.actor,'model/ddpg_actor_iter_%d.pth' %(update_count+1))
            print('%d: running_score:%.2f, ' %(update_count+1, running_score))
          self.update_ddpg(self.replay_memory.get_sample(batch_size))
          update_count += 1
          self.iteration += 1

        s0 = s1
        state = next_state
        score += reward
      if running_score == None:
        running_score = score
      else:
        running_score = running_score * 0.9 + score * 0.1


In [91]:
env = gym.make('Pendulum-v0')
agent = DDPG(env)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [92]:
# agent.train(env, update_limit=1, batch_size=2, update_per_episode=1, lr=1e-5, checkpoint=1)

In [93]:
agent.is_training = True
agent.train(env, update_limit=20000, batch_size=64, lr_actor=1e-4, lr_critic=1e-3, checkpoint=1000)

1000: running_score:-1187.10, 
2000: running_score:-1147.86, 
3000: running_score:-1038.53, 
4000: running_score:-1065.64, 
5000: running_score:-956.23, 
6000: running_score:-893.80, 
7000: running_score:-906.27, 
8000: running_score:-829.95, 
9000: running_score:-879.97, 
10000: running_score:-724.52, 
11000: running_score:-529.79, 
12000: running_score:-505.88, 
13000: running_score:-373.68, 
14000: running_score:-312.37, 
15000: running_score:-259.21, 
16000: running_score:-215.04, 
17000: running_score:-199.45, 
18000: running_score:-209.75, 
19000: running_score:-203.33, 
20000: running_score:-241.67, 


In [95]:
# use the final mean as solution and run a sample episode
agent.is_training = False
load_torch_model(agent.actor,'model/ddpg_actor_iter_16000.pth')
state = env.reset()
frames = []
frames.append(env.render(mode='rgb_array'))
ended = False
score = 0
while not ended:
  action = agent.pick_action(FloatTensor([state]).view(-1))
  (state, reward, ended, info) = env.step([action*2])
  score += reward
  frames.append(env.render(mode='rgb_array'))
print(score)

-129.79144899953593


In [96]:
%%capture
def animate(frames):
  fig, ax = plt.subplots()
  ax.grid('off')
  ax.axis('off')
  ims = []
  for i in range(len(frames)):
      im = plt.imshow(frames[i], animated=True)
      ims.append([im])
  ani = animation.ArtistAnimation(fig, ims, interval=20, blit=True, repeat_delay=1000)
  return ani

ani = animate(frames)
ani.save('pendulum_ddpg.mp4')

In [97]:
%%HTML
<video width="400" controls loop>
  <source src="pendulum_ddpg.mp4" type="video/mp4">
</video>